## Continuing from: scenario generation

In [1]:
from download_listing import get_listing
from mortgage import mortgage
from home_value import home_value, letting_cash_flow

In [2]:
from charts import outcomes_chart, pygal_outcomes_chart, wealth_chart, show

Loading BokehJS ...

In [3]:
import pandas as pd
import numpy as np
def scenario(
        price=500_000,
        transaction_cost = 0.05,
        downpayment = 0.05,
        years = 20,
        inflation = .018,
        inflation_houseprice = 0.045,
        rate_mortgage = 0.028,
        rental_rate = 0,
        interest_only = False
        ):
    
    scenario_conditions = [
        f'House price: £{price}',
        f'transaction_cost: £{transaction_cost*price:.0f} ({100*transaction_cost:.2f}%)',
        f'downpayment: £{downpayment*price:.0f} ({100*downpayment:.2f}%)',
        f'term (years): {years}',
        f'inflation {100*inflation:.2f}%',
        f'house price appreciation: {100*inflation_houseprice:.2f}%',
        f'rate_mortgage: {100*rate_mortgage:.2f}%',
        f'monthly rental income: £{rental_rate:.2f}',
        f'interest-only: {"yes" if interest_only else "no"}'
    ]
    transaction_cost *= price
    downpayment *= price 

    # Schedule and outcomes over time
    loan_schedule = mortgage(price, downpayment, rate_mortgage, years, interest_only=interest_only)
    debt_balance = loan_schedule.debt_balance.resample('Y').last()
    interest_paid = loan_schedule.interest_payments.resample('Y').sum().cumsum()
    home = home_value(price, inflation_houseprice, years)
    letting_income = letting_cash_flow(rental_rate, years, inflation).cumsum()
    cash_flow = (
        loan_schedule.full_payments.resample('Y').sum()
        + letting_income
    )
    cash_flow.iloc[0] -= transaction_cost
    cash_flow.iloc[0] -= downpayment
    cash_flow = cash_flow.cumsum()
    wealth = home + letting_income - debt_balance - transaction_cost

    # discounted value at time T

    schedule_df = pd.DataFrame(
        data = [debt_balance, interest_paid, cash_flow, wealth],
        index = 'debt_balance, interest_payments, cash_flow, wealth'.split(', ')
    ).T.dropna()
    discounting_factor = np.pv(inflation, np.arange(len(schedule_df)), 0, fv=-1)
    schedule_df = schedule_df.mul(discounting_factor, axis=0)
    
    outcomes_c = outcomes_chart(schedule_df)
    wealth_c = wealth_chart(schedule_df)
    
    return schedule_df, scenario_conditions, outcomes_c, wealth_c

## Make a pretty layout!

In [4]:
from IPython import display
import ipywidgets
import ipyvuetify as v
import traitlets
from traitlets import Int, Float, Unicode, Bool

In [5]:
# view
price_slider = v.Slider(v_model=250_000, 
                        min=50_000, max=1_000_000, 
                        step=10_000, 
                        class_='pa-3'
                       )
price_field = v.TextField(v_model=250_000, solo=True, class_='pa-3')
term_slider = v.Slider(v_model=20, min=5, max=30, step=5, thumb_label='always', class_='pa-3')
interest_only_checkbox = v.Checkbox(v_model=False, label='interest-only loan')


controls_box = v.Flex(d_flex=True, xs12=True, class_='ma-5', children=[
    v.Layout(row=True, children=[
        v.Flex(xs4=True, children=[v.Subheader(children=['Price'])]),
        v.Layout(xs8=True, row=True, wrap=True, children=[
            v.Flex(xs3=True, children=[price_field]),
            v.Flex(children=[price_slider])
        ])
    ]),
    v.Layout(row=True, children=[
        v.Flex(xs4=True, children=[v.Subheader(children=['Term(years)'])]),
        v.Flex(xs8=True, children=[term_slider])
    ]),
    v.Layout(row=True, children=[
        v.Flex(xs4=True, children=[v.Subheader(children=['Repayment'])]),
        v.Flex(xs8=True, children=[interest_only_checkbox])
    ])
])
outcomes_chart_box = v.Flex(d_flex=True, xs12=True, class_='ma-5', children=[])
wealth_chart_box = v.Flex(d_flex=True, xs12=True, class_='ma-5', children=[])
conditions_infobox = v.Flex(d_flex=True, xs12=True, class_='ma-5', children=[])

app = v.Container(fluid=True, grid_list_md=True, children=[
    v.Layout(row=True, wrap=True, children=[
        controls_box, conditions_infobox,
        outcomes_chart_box, wealth_chart_box
    ]),
])

# model
class Model(traitlets.HasTraits):
    price = Int(500_000)
    transaction_cost = Float(0.05)
    downpayment = Float(0.05)
    years = Int(20)
    inflation = Float(.018)
    inflation_houseprice = Float(0.045)
    rate_mortgage = Float(0.028)
    rental_rate = Int(0)
    interest_only = Bool(False)
    
model = Model()
    
# control

def render_scenario(change):
    # print(change.new)
    _, scenario_conditions, outcomes_c, wealth_c = scenario(
        downpayment=model.downpayment,
        inflation=model.inflation,
        inflation_houseprice=model.inflation_houseprice,
        interest_only=model.interest_only,
        price=model.price,
        rate_mortgage=model.rate_mortgage,
        rental_rate=model.rental_rate,
        transaction_cost=model.transaction_cost,
        years=model.years
    )
    
    conditions_infobox.children = [
        v.List(children=[
            v.ListItem(children=[line])
            for line in scenario_conditions
        ])
    ]

    o = ipywidgets.Output()
    outcomes_chart_box.children = [o]
    with o:
        show(outcomes_c)
        
    o = ipywidgets.Output()
    wealth_chart_box.children = [o]
    with o:
        show(wealth_c)
    

model.observe(render_scenario, traitlets.All)

ipywidgets.jslink((price_slider, 'v_model'), (price_field, 'v_model'))

traitlets.link(
    (model, 'price'), (price_slider, 'v_model')
)
traitlets.link(
    (model, 'years'), (term_slider, 'v_model')
)
traitlets.link(
    (model, 'interest_only'), (interest_only_checkbox, 'v_model')
);
render_scenario('')

In [6]:
app

Container(children=[Layout(children=[Flex(children=[Layout(children=[Flex(children=[Subheader(children=['Price…

## Exercise: Add a slider!